## Pour fonctionner dans googlecolab : <br>
$\renewcommand{\ket}[1]{| #1 \rangle}$  <!Cette commande n'est utile que dans google colab mais ne mange pas de pain-->
décommenter les deux cellules suivantes<br>
les exécuter<br>
redémarrer l'environnement d'exécution <br>

In [ ]:
!pip install myqlm

In [ ]:
!python -m qat.magics.install

Ce notebook reprend des exercices du début du livre d'ARNAUD BODIN <BR>

## QUANTUM : UN PEU DE MATHÉMATIQUES POUR L’INFORMATIQUE QUANTIQUE<br>


https://exo7math.github.io/quantum-exo7/


La liste des portes est <a href=https://myqlm.github.io/02_user_guide/01_write/01_digital_circuit/05_aqasm.html#list-of-gates>ici</a>

In [ ]:
import numpy as np
# everything we need to write a quantum circuit : 
#il vaut mieux importer element par element 
from qat.lang import Program,AbstractGate,X,H,CNOT,QRoutine,CCNOT


Préparation d'un état $$\phi>=\alpha\ket{0}+\beta\ket{1}$$

In [ ]:
Of = AbstractGate("Of", [], arity=1)
alpha=1./2
beta=(-3)**0.5/2
def Of_generator():
    # Fonction pour initialiser avec des coefficients donnés
    return np.array([[alpha, beta],
                     [-beta, alpha]])
Of.set_matrix_generator(Of_generator)   
prog = Program()
qbits = prog.qalloc(1)
prog.apply(Of(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
print("amplitudes (alpha,beta)=",alpha,beta)
print("probabilités (|alpha|^2,|beta|^2)=",np.abs(alpha)**2,np.abs(beta)**2)
#

Une autre manière d'initialiser un qbit $|\phi>=\alpha|0>+\beta|1>$ avec $|\alpha|^2+|\beta|^2=1$ est d'exprimer
$\alpha=\cos(\theta)e^{ia}+\sin(\theta)e^{ib}$ avec $\theta$, $a$ et $b$ réels.

$$\ket{\phi}=\cos(\theta)e^{ia}|0>+\sin(\theta)e^{ib}|1>.$$


On peut utiliser les portes préprogrammées $$PH(\theta)=\begin{pmatrix}1&0\\0&e^{i\theta}\end{pmatrix}$$ (phase shift)et 
$$R_Y(\theta)=\begin{pmatrix}\cos(\theta/2)&-\sin(\theta/2)\\ \sin(\theta/2)&\cos(\theta/2)\end{pmatrix}$$

Voir https://myqlm.github.io/02_user_guide/01_write/01_digital_circuit/05_aqasm.html


Exercice : Programmer le circuit

In [ ]:
from qat.lang import RY,PH
a=np.angle(alpha)
r1=np.abs(alpha)
b=np.angle(beta)
r2=np.abs(beta)
print("check modulus ",r1**2+r2**2)
theta=np.arctan(r2/r1)
prog = Program()
qbits = prog.qalloc(1)  # ici qbits[0] est initialisé à |0>
## appliquer les portes nécessaires pour calculer |phi> à partir de |0>


In [ ]:
#solution 
a=np.angle(alpha)
r1=np.abs(alpha)
b=np.angle(beta)
r2=np.abs(beta)
print("check modulus ",r1**2+r2**2)
theta=np.arctan(r2/r1)
prog = Program()
qbits = prog.qalloc(1)  # ici qbits[0] est initialisé à |0>
## appliquer les portes nécessaires pour calculer |phi> à partir de |0>
prog.apply(RY(-2*theta),qbits)
prog.apply(PH(b),qbits)
prog.apply(X,qbits)
prog.apply(PH(a),qbits)
prog.apply(X,qbits)

circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
print("amplitudes (alpha,beta)=",alpha,beta)
print("probabilités (|alpha|^2,|beta|^2)=",np.abs(alpha)**2,np.abs(beta)**2)


Programmation de la porte $$\sqrt{NOT}=\begin{pmatrix}1+i&1-i\\1-i&1+i\end{pmatrix}$$ 
1ere methode : avec un générateur


2eme methode : avec un circuit équivalent

$$\begin{array}{l}\ket{0}.......\mbox{SQRTNOT}......(1+i)\ket{0}+(1-i)\ket{1}\\
\ket{1}.......\mbox{SQRTNOT}......(1-i)\ket{0}+(1+i)\ket{1}\end{array}$$


In [ ]:
#programmer les deux  méthodes. Vérifier qu'elles donnent la même chose sur |0> et |1>





In [ ]:
#Proposition de solution 
#méthode 1:
SQRTNOT1 = AbstractGate("SQRTNOT", [], arity=1)
def SQRTNOT_generator():
    # Fonction pour initialiser avec des coefficients donnés
    return np.array([[1+1j, 1-1j],
                     [1-1j, 1+1j]])/2
SQRTNOT1.set_matrix_generator(SQRTNOT_generator)   

# méthode 2
from qat.lang import H,S
rout = QRoutine()
wires = rout.new_wires(1)
rout.apply(H,wires[0])
rout.apply(S,wires[0])
rout.apply(H,wires[0])
%qatdisplay --svg rout



# utilisation dans le circuit :
prog1 = Program()
qbit = prog1.qalloc(1)
prog1.apply(SQRTNOT1(), qbit)
circuit = prog1.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result1 = PyLinalg().submit(circuit.to_job())
prog2 = Program()
qbit = prog2.qalloc(1)
prog2.apply(rout, qbit)
circuit = prog2.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result2 = PyLinalg().submit(circuit.to_job())


print("Image of |0> : ")

for sample in result1:
    print("methode 1: State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
for sample in result2:
    print("methode 2 : State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
from qat.lang import RY,PH


# utilisation dans le circuit :
prog1 = Program()
qbit = prog1.qalloc(1)
prog1.apply(X,qbit)
prog1.apply(SQRTNOT1(), qbit)
circuit = prog1.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result1 = PyLinalg().submit(circuit.to_job())
prog2 = Program()
qbit = prog2.qalloc(1)
prog2.apply(X, qbit)
prog2.apply(rout, qbit)
circuit = prog2.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result2 = PyLinalg().submit(circuit.to_job())


print("Image of |1> :" )
for sample in result1:
    print("methode 1: State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
for sample in result2:
    print("méthode 2: State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
from qat.lang import RY,PH


Pour l'entrée $\ket{0}$ que donne une mesure après une porte $\sqrt{NOT}$ ? même question avec l'entrée $\ket{1}$ 

In [ ]:
#Proposition de solution
#Question 1
prog = Program()
qbits = prog.qalloc(1)
print("on applique SQRTNOT à |O>")
prog.apply(SQRTNOT1(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
# measuring qubit #0 50 times
result = PyLinalg().submit(circuit.to_job(qubits=[0], nbshots=1000))
for res in result:
    print("Qubit 0 has value %s (the probability of getting this result is %s +/- %s)"%(int(res.state[0]),
                                                                                 res.probability, res.err))
prog = Program()
qbits = prog.qalloc(1)
print("on applique SQRTNOT à |1>")
prog.apply(X, qbits)
prog.apply(SQRTNOT1(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
# measuring qubit #0 50 times
result = PyLinalg().submit(circuit.to_job(qubits=[0], nbshots=1000))
for res in result:
    print("L'image du Qubit 0 has value %s (the probability of getting this result is %s +/- %s)"%(int(res.state[0]),
                                                                                 res.probability, res.err))


Pour l'entrée $\dfrac{1}{2}\ket{0}+i\dfrac{\sqrt{3}}{2}\ket{1}$ que donne la sortie après la porte
 $\sqrt{NOT}$ ?  Que donne ensuite une
mesure ?

In [ ]:
#proposition de solution
print("alpha^2=",np.abs(alpha)**2," beta^2=",np.abs(beta)**2)
alpha=1./2
beta=(-3)**0.5/2

#Question 2
prog = Program()
qbits = prog.qalloc(1)
print("on applique SQRTNOT à |phi>")
prog.apply(Of(), qbits)
prog.apply(SQRTNOT1(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
# measuring qubit #0 50 times
result = PyLinalg().submit(circuit.to_job(qubits=[0], nbshots=1000))
for res in result:
    print("Qubit 0 has value %s (the probability of getting this result is %s +/- %s)"%(int(res.state[0]), res.probability, res.err))
       

Montrer que $$\sqrt{NOT}\sqrt{NOT}=NOT.$$

In [ ]:
#proposition de solution                                                                        
#Question 3
prog = Program()
qbits = prog.qalloc(1)
print("on applique 2 fois SQRTNOT à |0>")
prog.apply(SQRTNOT1(), qbits)
prog.apply(SQRTNOT1(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
# measuring qubit #0 50 times
result = PyLinalg().submit(circuit.to_job(qubits=[0], nbshots=1000))
for res in result:
    print("Qubit has value %s (the probability of getting this result is %s +/- %s)"%(int(res.state[0]),
                                                                                 res.probability, res.err))

prog = Program()
qbits = prog.qalloc(1)
print("on applique 2 fois SQRTNOT à |1>")
prog.apply(X, qbits)
prog.apply(SQRTNOT1(), qbits)
prog.apply(SQRTNOT1(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
# measuring qubit #0 50 times
result = PyLinalg().submit(circuit.to_job(qubits=[0], nbshots=1000))
for res in result:
    print("Qubit  has value %s (the probability of getting this result is %s +/- %s)"%(int(res.state[0]),
                                                                                 res.probability, res.err))

Initialisation d'un 2-qubit de norme 1 (Exemple page 10 [quantum book])
$$|\phi〉 =\dfrac{1}{\sqrt{6}}|0.0〉 +
\dfrac{i}{\sqrt{6}}
|1.0〉 +
\dfrac{1 + i}{\sqrt{3}}|1.1〉$$
Préparer l'état $\phi$ dans un 2-qbits avec une  AbstractGate 

In [ ]:
#proposition de solution
#Exemple page 10
#initialisation d'un 2-qubit de norme 1
Of = AbstractGate("Of", [], arity=2)
alpha=1/6**0.5
beta=1j/6**0.5
gamma=(1+1j)/3**0.5
def Of_generator():
    # Fonction pour initialiser avec des coefficients donnés
    return np.array([[alpha,0,0,0],
                     [0,0,0,0],
                     [beta,0,0,0],
                     [gamma,0,0,0]])
Of.set_matrix_generator(Of_generator)   

prog = Program()
qbits = prog.qalloc(2)
prog.apply(Of(), qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
print("amplitudes=",alpha,beta,gamma)
print("probabilités=",np.abs(alpha)**2,np.abs(beta)**2,np.abs(gamma)**2)

Préparer l'état de Bell
$$\ket{\phi^+}=\dfrac{1}{\sqrt{2}}(\ket{0.0}+\ket{1.1})$$
avec un circuit

In [ ]:
#proposition de solution
#page 11 Etat de Bell
prog = Program()
qbits = prog.qalloc(2)
prog.apply(H, qbits[0])
prog.apply(CNOT, qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))


Appliquez le circuit précédent à l'état $\ket{1.0}$ (au lieu de $\ket{0.0}$  pour obtenir l'état de BEll). Qu'obtenez-vous ?

Que faut-il insérer dans le circuit pour obtenir $$\ket{\phi^+}=\dfrac{1}{\sqrt{2}}(\ket{1.0}>+\ket{1.0})$$
 à partir de $\ket{0.0}$

In [ ]:
#proposition de solution
#exercice page 12
prog = Program()
qbits = prog.qalloc(2)
# on veut partir de l'état initial |1.0> au lieu de |0.0> donc on applique X à au 1er qubit
prog.apply(X, qbits[0])
prog.apply(H, qbits[0])
prog.apply(CNOT, qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
print("sortie à partir de |10>")
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
prog = Program()
qbits = prog.qalloc(2)
# Question : où insérer une porte X pour que le circuit produise (|0.1>+|1.0>)/sqrt(2)
prog.apply(X, qbits[1])
prog.apply(H, qbits[0])
prog.apply(CNOT, qbits)
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
print("obtention de  (|10>+|01>)/sqrt(2)")
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))


Montrons que deux circuits  sur des 2-qbits sont équivalents. On va les mettre dabns 2 qroutines et les tester sur les 4 etats de base


In [ ]:
# 1er circuit
rout1 = QRoutine()
wires = rout1.new_wires(2)
rout1.apply(H,wires[0])
rout1.apply(H,wires[1])
rout1.apply(CNOT,wires)  
rout1.apply(H,wires[0])
rout1.apply(H,wires[1])
%qatdisplay --svg rout1


In [ ]:
# 2ecircuit
rout2 = QRoutine()
wires = rout2.new_wires(2)
rout2.apply(CNOT,wires[1],wires[0])  
%qatdisplay --svg rout2


In [ ]:
from qat.qpus import PyLinalg
prog1 = Program()
qbits1 = [prog1.qalloc(2) for i in range(4)]
# au départ les 4 2-qbits1 contiennent |00>
prog1.apply(X,qbits1[1][0])  #condition initiale |10> dans le 2-qbits1[1]
prog1.apply(X,qbits1[2][1])  #condition initiale |01> dans le 2-qbits1[2]
prog1.apply(X,qbits1[3][0])  #condition initiale |11> dans le 2-qbits1[3]
prog1.apply(X,qbits1[3][1])
prog2 = Program()
qbits2 = [prog2.qalloc(2) for i in range(4)]
prog2.apply(X,qbits2[1][0])
prog2.apply(X,qbits2[2][1])
prog2.apply(X,qbits2[3][0])
prog2.apply(X,qbits2[3][1])
for i in range(4):
    prog1.apply(rout1, qbits1[i])
    circuit = prog1.to_circ(box_routines=True)
    result = PyLinalg().submit(circuit.to_job())
    for sample in result:
        print("Circuit 1 : State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
    prog1.apply(rout2, qbits2[i])
    circuit = prog2.to_circ(box_routines=True)
    result = PyLinalg().submit(circuit.to_job())
    for sample in result:
        print("Circuit 2 : State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))


Exercice Porte  de Toffoli<br>
On suppose que les qubits en entrée sont :<br>
$\ket{\psi_1}=\ket{0}+\ket{1}$<br>
$\ket{\psi_2}=\ket{0}+2i\ket{1}$<br>
$\ket{\psi_3}=2\ket{0}-3\ket{1}$<br>
Modifier le programme suivant pour calculer les trois qubits de sortie après une pporte de Toffoli.

In [ ]:
from qat.qpus import PyLinalg
prog = Program()
qbits = prog.qalloc(3)

prog.apply(CCNOT, qbits)  
circuit = prog.to_circ(box_routines=True)
%qatdisplay --svg circuit
from qat.qpus import PyLinalg
result = PyLinalg().submit(circuit.to_job())
for sample in result:
    print("State %s, amplitude %s, probability %s"%(sample.state, sample.amplitude, sample.probability))
